In [2]:
pip install sklearn-contrib-py-earth

Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-contrib-py-earth-0.1.0.tar.gz (1.0 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for sklearn-contrib-py-earth
Failed to build sklearn-contrib-py-earth
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [116 lines of output]
      c:\ProgramData\Anaconda3\Lib\site-packages\setuptools\dist.py:476: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      
              By 2024-Sep-26, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              ********************************************************************************
      
      !!
        opt = self.warn_dash_deprecation(opt, section)
      c:\

In [1]:
# Importações
import os
import warnings
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.svm import SVR
from prophet import Prophet
import matplotlib.pyplot as plt
import optuna
from joblib import Parallel, delayed
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from pyearth import Earth

# Configurações de gráficos e warnings
plt.style.use('fivethirtyeight')
warnings.filterwarnings("ignore")

# Função para carregar dados de múltiplos arquivos Excel
def carregar_dados(pasta, amostra=False):
    dados_finais = []
    for root, _, files in os.walk(pasta):
        for file in files:
            caminho = os.path.join(root, file)
            print(f"Carregando: {caminho}")
            f = pd.ExcelFile(caminho)
            for sheet in f.sheet_names:
                df = f.parse(sheet)
                df['pasta'] = file
                if amostra:
                    df = df.sample(frac=0.25, random_state=42)
                dados_finais.append(df)
    return pd.concat(dados_finais, ignore_index=True)

# Função para pré-processar os dados
def preprocessar_dados(df):
    if df['windspeed'].dtype == object:
        df['windspeed'] = df['windspeed'].str.replace(',', '.').astype(float)
    df['velocidade_vento'] = df['windspeed']
    df['datetime'] = pd.to_datetime(df['datetime'])
    Q1 = df['velocidade_vento'].quantile(0.25)
    Q3 = df['velocidade_vento'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df['velocidade_vento'] >= (Q1 - 1.5 * IQR)) & (df['velocidade_vento'] <= (Q3 + 1.5 * IQR))]
    df['velocidade_vento'] = df['velocidade_vento'].interpolate()
    return df

# Função auxiliar para garantir que os tamanhos de y_predito e y_verdadeiro sejam iguais
def ajustar_tamanhos(y_predito, y_verdadeiro):
    if len(y_predito) > len(y_verdadeiro):
        y_predito = y_predito[-len(y_verdadeiro):]
    elif len(y_verdadeiro) > len(y_predito):
        y_verdadeiro = y_verdadeiro[-len(y_predito):]
    return y_predito, y_verdadeiro

# Função genérica para rodar modelos
def rodar_modelos_paises(df, paises, modelo_func, n_splits=5, periodos=365):
    resultados_final = Parallel(n_jobs=-1)(delayed(modelo_func)(df, pais, n_splits, periodos) for pais in paises)
    return pd.DataFrame(resultados_final)

# Funções para otimizar e rodar SARIMAX
def otimizar_parametros_sarimax(trial, ts):
    pdq = trial.suggest_categorical('pdq', [(0, 0, 0), (1, 1, 1), (1, 0, 1), (0, 1, 1), (1, 1, 0)])
    seasonal_pdq = trial.suggest_categorical('seasonal_pdq', [(0, 0, 0, 12), (1, 1, 1, 12), (1, 0, 1, 12), (0, 1, 1, 12), (1, 1, 0, 12)])
    try:
        mod = SARIMAX(ts['y'], order=pdq, seasonal_order=seasonal_pdq, enforce_stationarity=False, enforce_invertibility=False)
        resultados = mod.fit(disp=False)
        return resultados.aic
    except Exception as e:
        print(f"Erro com {pdq}, {seasonal_pdq}: {e}")
        return float('inf')

def rodar_sarimax_pais(df, pais, n_splits, periodos):
    scaler = MinMaxScaler()
    tss = TimeSeriesSplit(n_splits=n_splits)
    print(f"Analisando país: {pais}")
    db = df[df['pasta'] == pais]
    ts = db[['datetime', 'velocidade_vento']].rename(columns={"datetime": 'ds', "velocidade_vento": "y"})
    ts['y'] = ts['y'].astype(float)
    ts = ts.sort_values('ds').set_index('ds')
    ts.index = pd.to_datetime(ts.index)
    ts = ts.resample('M').mean().reset_index()
    ts = ts.dropna()

    ts_escalado = scaler.fit_transform(ts[['y']])
    ts_escalado = pd.DataFrame(ts_escalado, index=ts.index, columns=['y'])

    estudo = optuna.create_study(direction='minimize')
    estudo.optimize(lambda trial: otimizar_parametros_sarimax(trial, ts_escalado), n_trials=50)
    melhor_param = estudo.best_params

    mod = SARIMAX(ts_escalado, order=melhor_param['pdq'], seasonal_order=melhor_param['seasonal_pdq'])
    resultados = mod.fit()

    predicao = resultados.get_forecast(steps=n_splits)
    y_predito = predicao.predicted_mean
    y_verdadeiro = ts_escalado['y'][-n_splits:]

    y_predito, y_verdadeiro = ajustar_tamanhos(y_predito, y_verdadeiro)

    mse_teste = mean_squared_error(y_verdadeiro, y_predito)
    mae_teste = mean_absolute_error(y_verdadeiro, y_predito)
    r2_teste = r2_score(y_verdadeiro, y_predito)

    return {
        'pais': pais,
        'modelo': 'SARIMAX',
        'MSE': mse_teste,
        'MAE': mae_teste,
        'R2': r2_teste
    }

# Funções para otimizar e rodar SVR
def otimizar_parametros_svr(trial, ts, tss):
    C = trial.suggest_loguniform('C', 1e-3, 1e3)
    gamma = trial.suggest_loguniform('gamma', 1e-4, 1e1)
    epsilon = trial.suggest_loguniform('epsilon', 1e-3, 1e1)
    svr = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)

    mse_scores = []
    for train_index, test_index in tss.split(ts):
        X_treino, X_teste = ts.index[train_index], ts.index[test_index]
        y_treino, y_teste = ts.iloc[train_index], ts.iloc[test_index]
        svr.fit(np.array(X_treino).reshape(-1, 1), y_treino)
        y_teste_predito = svr.predict(np.array(X_teste).reshape(-1, 1))
        mse_scores.append(mean_squared_error(y_teste, y_teste_predito))
    return np.mean(mse_scores)

def rodar_svr_pais(df, pais, n_splits, periodos):
    scaler = MinMaxScaler()
    tss = TimeSeriesSplit(n_splits=n_splits)
    print(f"Analisando país: {pais}")
    db = df[df['pasta'] == pais]
    ts = db[['datetime', 'velocidade_vento']].rename(columns={"datetime": 'ds', "velocidade_vento": "y"})
    ts['y'] = ts['y'].astype(float)
    ts = ts.sort_values('ds').set_index('ds')
    ts.index = pd.to_datetime(ts.index)
    ts = ts.resample('M').mean().reset_index()
    ts = ts.dropna()

    ts_escalado = scaler.fit_transform(ts[['y']])
    ts_escalado = pd.DataFrame(ts_escalado, index=ts.index, columns=['y'])

    estudo = optuna.create_study(direction='minimize')
    estudo.optimize(lambda trial: otimizar_parametros_svr(trial, ts_escalado, tss), n_trials=50)
    melhor_svr = SVR(kernel='rbf', **estudo.best_params)

    resultados = []

    for train_index, test_index in tss.split(ts_escalado):
        X_treino, X_teste = ts.index[train_index], ts.index[test_index]
        y_treino, y_teste = ts.iloc[train_index], ts.iloc[test_index]

        melhor_svr.fit(np.array(X_treino).reshape(-1, 1), y_treino)

        y_treino_predito = melhor_svr.predict(np.array(X_treino).reshape(-1, 1)).reshape(-1, 1)
        y_teste_predito = melhor_svr.predict(np.array(X_teste).reshape(-1, 1)).reshape(-1, 1)

        y_treino_predito = scaler.inverse_transform(y_treino_predito)
        y_teste_predito = scaler.inverse_transform(y_teste_predito)
        y_treino_orig = scaler.inverse_transform(y_treino.values.reshape(-1, 1))
        y_teste_orig = scaler.inverse_transform(y_teste.values.reshape(-1, 1))

        y_teste_predito, y_teste_orig = ajustar_tamanhos(y_teste_predito, y_teste_orig)

        mse_teste = mean_squared_error(y_teste_orig, y_teste_predito)
        mae_teste = mean_absolute_error(y_teste_orig, y_teste_predito)
        r2_teste = r2_score(y_teste_orig, y_teste_predito)

        resultados.append({
            'pais': pais,
            'modelo': 'SVR',
            'MSE': mse_teste,
            'MAE': mae_teste,
            'R2': r2_teste
        })

    return resultados

# Funções para otimizar e rodar Prophet
def otimizar_parametros_prophet(trial, ts):
    changepoint_prior_scale = trial.suggest_loguniform('changepoint_prior_scale', 0.001, 0.5)
    seasonality_prior_scale = trial.suggest_loguniform('seasonality_prior_scale', 0.01, 10.0)
    
    modelo = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale
    )
    modelo.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    modelo.fit(ts)

    futuro = modelo.make_future_dataframe(periods=365, freq='M')
    previsao = modelo.predict(futuro)
    y_predito = previsao['yhat'][-len(ts):].values
    y_verdadeiro = ts['y'].values

    return mean_squared_error(y_verdadeiro, y_predito)

def rodar_prophet_pais(df, pais, n_splits, periodos):
    print(f"Analisando país: {pais}")
    db = df[df['pasta'] == pais]
    ts = db[['datetime', 'velocidade_vento']].rename(columns={"datetime": 'ds', "velocidade_vento": "y"})
    ts['y'] = ts['y'].astype(float)
    ts = ts.sort_values('ds').dropna()

    estudo = optuna.create_study(direction='minimize')
    estudo.optimize(lambda trial: otimizar_parametros_prophet(trial, ts), n_trials=50)
    melhor_params = estudo.best_params

    m = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=melhor_params['changepoint_prior_scale'],
        seasonality_prior_scale=melhor_params['seasonality_prior_scale']
    )
    m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    m.fit(ts)

    futuro = m.make_future_dataframe(periods=periodos, freq='M')
    previsao = m.predict(futuro)

    y_predito = previsao['yhat'][-len(ts):].values
    y_verdadeiro = ts['y'].values

    y_predito, y_verdadeiro = ajustar_tamanhos(y_predito, y_verdadeiro)

    mse_teste = mean_squared_error(y_verdadeiro, y_predito)
    mae_teste = mean_absolute_error(y_verdadeiro, y_predito)
    r2_teste = r2_score(y_verdadeiro, y_predito)

    resultados_previsao = previsao[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    print(resultados_previsao)

    m.plot(previsao)
    plt.title(f"Previsões - {pais}")
    plt.show()

    m.plot_components(previsao)
    plt.show()

    return {
        'pais': pais,
        'modelo': 'Prophet',
        'MSE': mse_teste,
        'MAE': mae_teste,
        'R2': r2_teste
    }

# Classe para encapsular o modelo MARS
class MARSModel(BaseEstimator, RegressorMixin):
    def __init__(self, degree=2):
        self.degree = degree
        self.model = None

    def fit(self, X, y):
        self.model = Earth(max_degree=self.degree)
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

# Funções para otimizar e rodar MARS
def otimizar_parametros_mars(trial, ts, tss):
    degree = trial.suggest_int('degree', 1, 5)
    mars_model = MARSModel(degree=degree)
    
    mse_scores = []
    for train_index, test_index in tss.split(ts):
        X_treino, X_teste = ts.index[train_index], ts.index[test_index]
        y_treino, y_teste = ts.iloc[train_index], ts.iloc[test_index]
        mars_model.fit(np.array(X_treino).reshape(-1, 1), y_treino)
        y_teste_predito = mars_model.predict(np.array(X_teste).reshape(-1, 1))
        mse_scores.append(mean_squared_error(y_teste, y_teste_predito))
    return np.mean(mse_scores)

def rodar_mars_pais(df, pais, n_splits, periodos):
    scaler = MinMaxScaler()
    tss = TimeSeriesSplit(n_splits=n_splits)
    print(f"Analisando país: {pais}")
    db = df[df['pasta'] == pais]
    ts = db[['datetime', 'velocidade_vento']].rename(columns={"datetime": 'ds', "velocidade_vento": "y"})
    ts['y'] = ts['y'].astype(float)
    ts = ts.sort_values('ds').set_index('ds')
    ts.index = pd.to_datetime(ts.index)
    ts = ts.resample('M').mean().reset_index()
    ts = ts.dropna()

    ts_escalado = scaler.fit_transform(ts[['y']])
    ts_escalado = pd.DataFrame(ts_escalado, index=ts.index, columns=['y'])

    X = np.arange(len(ts_escalado)).reshape(-1, 1)
    y = ts_escalado['y'].values

    estudo = optuna.create_study(direction='minimize')
    estudo.optimize(lambda trial: otimizar_parametros_mars(trial, ts_escalado, tss), n_trials=50)
    melhor_mars = MARSModel(degree=estudo.best_params['degree'])

    resultados = []

    for train_index, test_index in tss.split(ts_escalado):
        X_treino, X_teste = X[train_index], X[test_index]
        y_treino, y_teste = y[train_index], y[test_index]

        melhor_mars.fit(X_treino, y_treino)

        y_treino_predito = melhor_mars.predict(X_treino).reshape(-1, 1)
        y_teste_predito = melhor_mars.predict(X_teste).reshape(-1, 1)

        y_treino_predito = scaler.inverse_transform(y_treino_predito)
        y_teste_predito = scaler.inverse_transform(y_teste_predito)
        y_treino_orig = scaler.inverse_transform(y_treino.reshape(-1, 1))
        y_teste_orig = scaler.inverse_transform(y_teste.reshape(-1, 1))

        y_teste_predito, y_teste_orig = ajustar_tamanhos(y_teste_predito, y_teste_orig)

        mse_teste = mean_squared_error(y_teste_orig, y_teste_predito)
        mae_teste = mean_absolute_error(y_teste_orig, y_teste_predito)
        r2_teste = r2_score(y_teste_orig, y_teste_predito)

        resultados.append({
            'pais': pais,
            'modelo': 'MARS',
            'MSE': mse_teste,
            'MAE': mae_teste,
            'R2': r2_teste
        })

    return resultados

# Carregar e pré-processar os dados
df = carregar_dados(pasta_dados)
df = preprocessar_dados(df)

# Definição dos países
paises = df['pasta'].unique()

# Executa a otimização e avaliação dos modelos
resultado_sarimax = rodar_modelos_paises(df, paises, rodar_sarimax_pais)
resultado_svr = rodar_modelos_paises(df, paises, rodar_svr_pais)
resultado_prophet = rodar_modelos_paises(df, paises, rodar_prophet_pais, periodos=365)
resultado_mars = rodar_modelos_paises(df, paises, rodar_mars_pais)

# Exibir resultados
print("Resultados SARIMAX:")
print(pd.DataFrame(resultado_sarimax))
print("\nResultados SVR:")
print(pd.DataFrame(resultado_svr))
print("\nResultados Prophet:")
print(pd.DataFrame(resultado_prophet))
print("\nResultados MARS:")
print(pd.DataFrame(resultado_mars))


ModuleNotFoundError: No module named 'pyearth'